# Fill Fitting 

This section performs mass fitting on individual Fill files for both **B2OC** (B⁺ → D⁰π⁺) and **B2CC** (B⁺ → J/ψK⁺) decay modes.

**Fitting Method**: Double-sided Crystal Ball PDFs for signal + exponential background

**Output**: 
- New `fit_results` TTree containing all fit parameters and uncertainties
- Canvas objects with mass plots showing data and fitted components

The workflow consists of three cells: cleanup, fitting loop, and visualization.

In [19]:
import ROOT as r
import numpy as np
import matplotlib.pyplot as plt
import math
from ROOT import RooFit
from ROOT import TChain
from ROOT import RooStats
from datetime import datetime
from uncertainties import ufloat
from scipy.optimize import curve_fit
from pathlib import Path

# Define data directories for organized file management
DATA_CLEAN = Path("data/processed_clean_bp_p")   

plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
print(r.gROOT.GetVersion())

%jsroot on

c = r.TCanvas()

6.34.04


In [ ]:
from array import array                     # ROOT TTrees need C-style arrays
import re                               # regular expressions for filename parsing
r.EnableImplicitMT()                    # enable multi-threading in RooFit

# Input files to process (fill-level B⁺ → D⁰π⁺ data)
files = [
    DATA_CLEAN/"2024_B2OC_B5_F10090.root", DATA_CLEAN/"2024_B2OC_B5_F10059.root", DATA_CLEAN/"2024_B2OC_B5_F10061.root",
    DATA_CLEAN/"2024_B2OC_B5_F10066.root", DATA_CLEAN/"2024_B2OC_B5_F10069.root", DATA_CLEAN/"2024_B2OC_B5_F10070.root",
    DATA_CLEAN/"2024_B2OC_B5_F10072.root", DATA_CLEAN/"2024_B2OC_B5_F10073.root", DATA_CLEAN/"2024_B2OC_B5_F10074.root",
    DATA_CLEAN/"2024_B2OC_B5_F10075.root", DATA_CLEAN/"2024_B2OC_B5_F10077.root", DATA_CLEAN/"2024_B2OC_B5_F10082.root",
    DATA_CLEAN/"2024_B2OC_B5_F10084.root", DATA_CLEAN/"2024_B2OC_B5_F10086.root", DATA_CLEAN/"2024_B2OC_B5_F10087.root",
    DATA_CLEAN/"2024_B2OC_B5_F10090.root", DATA_CLEAN/"2024_B2OC_B5_F10091.root", DATA_CLEAN/"2024_B2OC_B5_F10092.root",
    DATA_CLEAN/"2024_B2OC_B5_F10093.root", DATA_CLEAN/"2024_B2OC_B5_F10094.root", DATA_CLEAN/"2024_B2OC_B5_F10095.root",
    DATA_CLEAN/"2024_B2OC_B5_F10096.root", DATA_CLEAN/"2024_B2OC_B5_F10097.root", DATA_CLEAN/"2024_B2OC_B5_F10098.root",
    DATA_CLEAN/"2024_B2OC_B5_F10099.root", DATA_CLEAN/"2024_B2OC_B5_F10100.root", DATA_CLEAN/"2024_B2OC_B6_F10104.root",
    DATA_CLEAN/"2024_B2OC_B6_F10105.root", DATA_CLEAN/"2024_B2OC_B6_F10107.root", DATA_CLEAN/"2024_B2OC_B6_F10112.root",
    DATA_CLEAN/"2024_B2OC_B6_F10115.root", DATA_CLEAN/"2024_B2OC_B6_F10116.root", DATA_CLEAN/"2024_B2OC_B6_F10117.root",
    DATA_CLEAN/"2024_B2OC_B6_F10118.root", DATA_CLEAN/"2024_B2OC_B6_F10122.root", DATA_CLEAN/"2024_B2OC_B6_F10125.root",
    DATA_CLEAN/"2024_B2OC_B6_F10126.root", DATA_CLEAN/"2024_B2OC_B6_F10127.root", DATA_CLEAN/"2024_B2OC_B6_F10130.root",
    DATA_CLEAN/"2024_B2OC_B6_F10132.root", DATA_CLEAN/"2024_B2OC_B6_F10134.root", DATA_CLEAN/"2024_B2OC_B6_F10137.root",
    DATA_CLEAN/"2024_B2OC_B6_F10138.root", DATA_CLEAN/"2024_B2OC_B7_F10197.root", DATA_CLEAN/"2024_B2OC_B7_F10199.root",
    DATA_CLEAN/"2024_B2OC_B7_F10200.root", DATA_CLEAN/"2024_B2OC_B7_F10201.root", DATA_CLEAN/"2024_B2OC_B7_F10204.root",
    DATA_CLEAN/"2024_B2OC_B7_F10206.root", DATA_CLEAN/"2024_B2OC_B7_F10207.root", DATA_CLEAN/"2024_B2OC_B7_F10208.root",
    DATA_CLEAN/"2024_B2OC_B7_F10209.root", DATA_CLEAN/"2024_B2OC_B7_F10210.root", DATA_CLEAN/"2024_B2OC_B7_F10211.root",
    DATA_CLEAN/"2024_B2OC_B7_F10213.root", DATA_CLEAN/"2024_B2OC_B8_F10214.root", DATA_CLEAN/"2024_B2OC_B8_F10215.root",
    DATA_CLEAN/"2024_B2OC_B8_F10216.root", DATA_CLEAN/"2024_B2OC_B8_F10218.root", DATA_CLEAN/"2024_B2OC_B8_F10219.root",
    DATA_CLEAN/"2024_B2OC_B8_F10222.root", DATA_CLEAN/"2024_B2OC_B8_F10223.root", DATA_CLEAN/"2024_B2OC_B8_F10225.root",
    DATA_CLEAN/"2024_B2OC_B8_F10226.root", DATA_CLEAN/"2024_B2OC_B8_F10230.root", DATA_CLEAN/"2024_B2OC_B8_F10232.root"
]

# Observable: B+ mass variable  
x = r.RooRealVar("Bp_DTF_OwnPV_MASS", "B^{+} mass", 5200, 5450)    # MeV/c² range

# B+ → D⁰π+ signal parameters (main peak around 5278 MeV)
mean = r.RooRealVar("mean", "mean", 5278.46, 5250, 5300)

# Crystal Ball 1 parameters
alpha = r.RooRealVar("alpha", "alpha", 1.25015, 1, 2)
n = r.RooRealVar("n", "n", 2.40748, 0.5, 5)
cb_sigma = r.RooRealVar("cb_sigma", "cb_sigma", 11, 1.0, 30)  
crystal_ball = r.RooCBShape("crystal_ball", "Crystal ball PDF", x, mean, cb_sigma, alpha, n)

# Background model
tau = r.RooRealVar("tau", "Decay constant", -0.00219122, -1, 0)  # exponential slope
background = r.RooExponential("background", "Exponential background", x, tau)

# Crystal Ball 2 parameters
alpha_2 = r.RooRealVar("alpha2", "alpha2", -2.2701, -20, -0.01)
n_2 = r.RooRealVar("n2", "n2", 2.47972, 0.05, 50)
cb_sigma_2 = r.RooRealVar("cb_sigma2", "cb_sigma2", 12.7184, 1.0, 20)   
crystal_ball_2 = r.RooCBShape("crystal_ball2", "Crystal ball PDF 2", x, mean, cb_sigma_2, alpha_2, n_2)

frac_cb_2 = r.RooRealVar("frac_cb_2", "Fraction of crystal ball 2", 0.65, 0.0, 1.0)

# Parameter constraints: fix shape parameters, float yields and resolutions
mean.setConstant(False)          # allow mean to float
alpha.setConstant(True)          # fix tail parameter
n.setConstant(True)              # fix tail parameter
cb_sigma.setConstant(True)       # fix resolution
tau.setConstant(False)           # allow background slope to float
alpha_2.setConstant(True)        # fix tail parameter
n_2.setConstant(True)            # fix tail parameter
cb_sigma_2.setConstant(False)    # allow resolution to float
frac_cb_2.setConstant(False)     # allow CB fraction to float

# Helper functions for branch management
def make_branch(tree, name, var_type='d'):
    """Helper function to create a branch with array buffer"""
    if var_type == 'd':
        buf = array('d', [0.])
        tree.Branch(name, buf, f"{name}/D")
    elif var_type == 'i':
        buf = array('i', [0])
        tree.Branch(name, buf, f"{name}/I")
    return buf

def setaddr(tree, name, buf): 
    """Helper function to set branch addresses safely"""
    branch = tree.GetBranch(name)
    if branch: 
        branch.SetAddress(buf)

# Main fitting loop over input files
for fname in files:

    print(f"\n▶ fitting fill file {fname}")

    # Open file and get tree
    f = r.TFile.Open(str(fname), "UPDATE")          # UPDATE keeps existing content
    tree = f.Get("ST-b2oc")
    if not tree:
        print("ST-b2oc tree not found, skipping");  f.Close();  continue

    # Load data into RooDataSet
    data = r.RooDataSet("data", "data", r.RooArgSet(x), r.RooFit.Import(tree))
    Nevt = data.numEntries()
    print(f"entries in tree = {Nevt}")

    # Define yield variables with initial estimates
    nsig = r.RooRealVar("nsig", "yield sig", Nevt * 0.25, Nevt * 0.05, Nevt * 0.4)   # ~25% signal
    nbkg = r.RooRealVar("nbkg", "yield bkg", Nevt * 0.75, Nevt * 0.6, Nevt * 0.95)   # ~75% background
    ntot = r.RooFormulaVar("ntot","@0+@1", r.RooArgList(nsig, nbkg))
    
    # Construct composite PDF
    sig_pdf = r.RooAddPdf("sig","", r.RooArgList(crystal_ball_2, crystal_ball,),
                                    r.RooArgList(frac_cb_2))

    # Total model: signal + background
    model = r.RooAddPdf("model","sig+bkg", r.RooArgList(sig_pdf, background),
                                         r.RooArgList(nsig,    nbkg))

    # Perform fit
    fit_res = model.fitTo(data, r.RooFit.Save(True), r.RooFit.Strategy(2))

    # Print fit results
    print("\n=== fitted yields ===")
    print(f"nsig  = {nsig.getVal():.0f} ± {nsig.getError():.0f}")
    print(f"nbkg  = {nbkg.getVal():.0f} ± {nbkg.getError():.0f}")
    print(f"ntot  = {ntot.getVal():.0f} (derived)")
    print(frac_cb_2.getVal())
    
    # Error propagation with uncertainties package
    sig_u = ufloat(nsig.getVal(), nsig.getError()) # background fraction, propagated uncertainty 
    bkg_u = ufloat(nbkg.getVal(), nbkg.getError())
    frac_bkg = bkg_u / (sig_u + bkg_u)

    print(f"background fraction = {frac_bkg:.6f}")
    print(f"fit status = {fit_res.status()}, covQual = {fit_res.covQual()}")

    # Store results in TTree branches
    f.cd()
    res_tree = f.Get("fit_results")
    
    # Create results tree if it doesn't exist
    if not res_tree:
        res_tree = r.TTree("fit_results", "Mass-fit results per fill")
        
        # Create branches using helper function
        br_mean           = make_branch(res_tree, "mean")
        br_mean_e         = make_branch(res_tree, "mean_err")
        br_alpha          = make_branch(res_tree, "alpha1")
        br_alpha_e        = make_branch(res_tree, "alpha1_err")
        br_n              = make_branch(res_tree, "n1")
        br_n_e            = make_branch(res_tree, "n1_err")
        br_cb_sigma       = make_branch(res_tree, "cb_sigma1")
        br_cb_sigma_e     = make_branch(res_tree, "cb_sigma1_err")
        br_tau            = make_branch(res_tree, "tau")
        br_tau_e          = make_branch(res_tree, "tau_err")
        br_alpha_2        = make_branch(res_tree, "alpha2")
        br_alpha_2_e      = make_branch(res_tree, "alpha2_err")
        br_n_2            = make_branch(res_tree, "n2")
        br_n_2_e          = make_branch(res_tree, "n2_err")
        br_cb_sigma_2     = make_branch(res_tree, "cb_sigma2")
        br_cb_sigma_2_e   = make_branch(res_tree, "cb_sigma2_err")
        br_frac_cb_2      = make_branch(res_tree, "frac_cb_2")
        br_frac_cb_2_e    = make_branch(res_tree, "frac_cb_2_err")
        br_Nevt           = make_branch(res_tree, "Nevt")          
        br_nsig           = make_branch(res_tree, "nsig")
        br_nsig_e         = make_branch(res_tree, "nsig_err")
        br_nbkg           = make_branch(res_tree, "nbkg")
        br_nbkg_e         = make_branch(res_tree, "nbkg_err")
        br_ntot           = make_branch(res_tree, "ntot")
        br_ntot_e         = make_branch(res_tree, "ntot_err")
        
        # Integer branches for fit quality
        br_status         = make_branch(res_tree, "status", 'i')
        br_covqual        = make_branch(res_tree, "covqual", 'i')
        
    else:
        # Tree exists: create arrays and set branch addresses using helper functions
        br_mean           = array('d', [0.]); br_mean_e         = array('d', [0.])
        br_alpha          = array('d', [0.]); br_alpha_e        = array('d', [0.])
        br_n              = array('d', [0.]); br_n_e            = array('d', [0.])
        br_cb_sigma       = array('d', [0.]); br_cb_sigma_e     = array('d', [0.])
        br_tau            = array('d', [0.]); br_tau_e          = array('d', [0.])
        br_alpha_2        = array('d', [0.]); br_alpha_2_e      = array('d', [0.])
        br_n_2            = array('d', [0.]); br_n_2_e          = array('d', [0.])
        br_cb_sigma_2     = array('d', [0.]); br_cb_sigma_2_e   = array('d', [0.])
        br_frac_cb_2      = array('d', [0.]); br_frac_cb_2_e    = array('d', [0.])
        br_Nevt           = array('d', [0.])
        br_nsig           = array('d', [0.]); br_nsig_e         = array('d', [0.])
        br_nbkg           = array('d', [0.]); br_nbkg_e         = array('d', [0.])
        br_ntot           = array('d', [0.]); br_ntot_e         = array('d', [0.])
        br_status         = array('i', [0]); br_covqual        = array('i', [0])

        # Set all branch addresses using dictionary approach with helper function
        branch_mapping = {
            "mean": br_mean, "mean_err": br_mean_e,
            "alpha1": br_alpha, "alpha1_err": br_alpha_e,
            "n1": br_n, "n1_err": br_n_e,
            "cb_sigma1": br_cb_sigma, "cb_sigma1_err": br_cb_sigma_e,
            "tau": br_tau, "tau_err": br_tau_e,
            "alpha2": br_alpha_2, "alpha2_err": br_alpha_2_e,
            "n2": br_n_2, "n2_err": br_n_2_e,
            "cb_sigma2": br_cb_sigma_2, "cb_sigma2_err": br_cb_sigma_2_e,
            "frac_cb_2": br_frac_cb_2, "frac_cb_2_err": br_frac_cb_2_e,
            "Nevt": br_Nevt,
            "nsig": br_nsig, "nsig_err": br_nsig_e,
            "nbkg": br_nbkg, "nbkg_err": br_nbkg_e,
            "ntot": br_ntot, "ntot_err": br_ntot_e,
            "status": br_status, "covqual": br_covqual
        }
        
        for branch_name, buffer in branch_mapping.items():
            setaddr(res_tree, branch_name, buffer)

    # Fill branch values with fit results
    br_mean[0]            = mean.getVal()
    br_mean_e[0]          = mean.getError()
    br_alpha[0]           = alpha.getVal()
    br_alpha_e[0]         = alpha.getError()
    br_n[0]               = n.getVal()
    br_n_e[0]             = n.getError()
    br_cb_sigma[0]        = cb_sigma.getVal()
    br_cb_sigma_e[0]      = cb_sigma.getError()
    br_tau[0]             = tau.getVal()
    br_tau_e[0]           = tau.getError()
    br_alpha_2[0]         = alpha_2.getVal()
    br_alpha_2_e[0]       = alpha_2.getError()
    br_n_2[0]             = n_2.getVal()
    br_n_2_e[0]           = n_2.getError()
    br_cb_sigma_2[0]      = cb_sigma_2.getVal()
    br_cb_sigma_2_e[0]    = cb_sigma_2.getError()
    br_frac_cb_2[0]       = frac_cb_2.getVal()
    br_frac_cb_2_e[0]     = frac_cb_2.getError()
    br_Nevt[0]            = Nevt
    br_nsig[0]            = nsig.getVal()
    br_nsig_e[0]          = nsig.getError()
    br_nbkg[0]            = nbkg.getVal()
    br_nbkg_e[0]          = nbkg.getError()
    
    ntot_u                = sig_u + bkg_u         
    br_ntot[0]            = ntot_u.n
    br_ntot_e[0]          = ntot_u.s
    br_status[0]          = fit_res.status()
    br_covqual[0]         = fit_res.covQual()

    # Store fit results in branch arrays
    res_tree.Fill()

    # Create and save fit plots
    canvas = r.TCanvas(f"canvas_{fname.stem}", "Dpi mass fit", 800, 600)
    frame = x.frame(r.RooFit.Title(f"Dpi mass fit: {fname.stem}"))

    # Plot data and model components
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kBlack), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.LineColor(r.kRed), r.RooFit.Name("total_curve"))
    model.plotOn(frame, r.RooFit.Components("background"), 
                r.RooFit.LineStyle(r.kDashed), r.RooFit.LineColor(r.kGreen+2))
    model.plotOn(frame, r.RooFit.Components("crystal_ball"), 
                r.RooFit.LineStyle(2), r.RooFit.LineColor(r.kBlue))
    model.plotOn(frame, r.RooFit.Components("crystal_ball2"), 
                r.RooFit.LineStyle(3), r.RooFit.LineColor(r.kCyan))

    # Format and draw the plot
    frame.SetTitle(f"B+ -> Dpi Mass Fit: {fname.stem}")
    frame.GetXaxis().SetTitle("M(B^{+})  [MeV/c^{2}]")
    frame.GetXaxis().CenterTitle(True)
    frame.GetYaxis().CenterTitle(True)
    frame.Draw()
    
    # Add legend for plot components
    legend = r.TLegend(0.72, 0.60, 0.98, 0.88)
    legend.SetTextSize(0.025)
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.AddEntry(frame.findObject("total_curve"), "Total fit", "l")
    legend.AddEntry(frame.findObject("h_data"), "Data", "lep")
    legend.AddEntry(0, "Background", "l")  
    legend.Draw()

    # Calculate automatic y-axis minimum for log scale
    x_min_bg = 5400 
    x_max_bg = 5450
    n_bins = 100
    bin_width = (x.getMax() - x.getMin()) / n_bins
    n_bg = data.reduce(f"{x.GetName()} >= {x_min_bg} && {x.GetName()} < {x_max_bg}").numEntries()
    ymin = max(1, n_bg / ((x_max_bg - x_min_bg) / bin_width))

    # Apply log scale and save PNG
    frame.SetMinimum(ymin * 0.8)
    canvas.SetLogy()
    canvas.Write(f"canvas_{fname.stem}_log")
    
    # Save log-scale PNG to output directory
    log_plot_path = DATA_CLEAN / f"dpi_fit_{fname.stem}_log.png"
    canvas.SetCanvasSize(2400/1.5, 1800/1.5)
    canvas.SaveAs(str(log_plot_path))
    print(f"saved log-scale plot: {log_plot_path}")

    # Save results and clean up
    res_tree.Write("", r.TObject.kOverwrite)
    f.Close()
    
    print(f"Completed processing {fname.name}\n")




▶ fitting fill file data/processed_clean_bp_p/2024_B2OC_B5_F10090.root
entries in tree = 166885

=== fitted yields ===
nsig  = 47389 ± 460
nbkg  = 119494 ± 532
ntot  = 166884 (derived)
0.8593511774589712
background fraction = 0.716033+/-0.002171
fit status = 0, covQual = 3
saved log-scale plot: data/processed_clean_bp_p/dpi_fit_2024_B2OC_B5_F10090_log.png
Completed processing 2024_B2OC_B5_F10090.root


▶ fitting fill file data/processed_clean_bp_p/2024_B2OC_B5_F10059.root
entries in tree = 23491

=== fitted yields ===
nsig  = 6783 ± 183
nbkg  = 16709 ± 208
ntot  = 23492 (derived)
0.8053006779273195
background fraction = 0.711273+/-0.006100
fit status = 0, covQual = 3
saved log-scale plot: data/processed_clean_bp_p/dpi_fit_2024_B2OC_B5_F10059_log.png
Completed processing 2024_B2OC_B5_F10059.root


▶ fitting fill file data/processed_clean_bp_p/2024_B2OC_B5_F10061.root
entries in tree = 71877

=== fitted yields ===
nsig  = 20407 ± 305
nbkg  = 51470 ± 352
ntot  = 71877 (derived)
0.7295805

KeyboardInterrupt: 

[#1] INFO:DataHandling -- RooAbsReal::attachToTree(Bp_DTF_OwnPV_MASS) TTree Float_t branch Bp_DTF_OwnPV_MASS will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #1 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5769.78
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #2 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5113.36
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #3 because Bp_DTF_OwnPV_MASS cannot accommodate the value 6008.26
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #4 because Bp_DTF_OwnPV_MASS cannot accommodate the value 4921.33
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping ...
[#0] WARNING:DataHandling -- RooTreeDataStore::loadValues(data) Ignored 521656 out-of-range events
[#1] INFO:Minimization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -937254.4452 Edm =       542.7562925 NCalls =     33
Warning in <Minuit2>: MnPosDef Matrix forced pos-def by adding to diagonal 0.0481434
Info in <Minuit2>: MnSeedGenerator run Hesse - Initial seeding state: 
  Minimum value : -937254.4452
  Edm           : 130867.5998
  Internal parameters:	[     0.2356918764      0.304692654     0.1388456842    -0.1433475689     0.1433475689      1.477141165]	
  Internal gradient  :	[     -1972.139386     -247.1339835     -741.6681686     -178.9578019      536.8702527     -19239.32038]	
  Internal covariance matrix:
[[     0.05829643     0.32089942  -0.0069620644   -0.018649869     0.01940187 -4.2474562e-05]
 [     0.32089942      1.7768683   -0.038534769    -0.10250174     0.10663422 -0.00025790075]
 [  -0.0069620644   -0.038534769  0.00087475508   0.0022252763  -0.0023149911  4.7870663e-06]
 [   -0.018649

In [ ]:
from array import array                     # ROOT TTrees need C-style arrays
import re                               # regular expressions for filename parsing
r.EnableImplicitMT()                    # enable multi-threading in RooFit

# Input files to process (fill-level B⁺ → J/ψK⁺ data)
files = [
    DATA_CLEAN/"2024_B2CC_B5_F10090.root", DATA_CLEAN/"2024_B2CC_B5_F10059.root", DATA_CLEAN/"2024_B2CC_B5_F10061.root",
    DATA_CLEAN/"2024_B2CC_B5_F10066.root", DATA_CLEAN/"2024_B2CC_B5_F10069.root", DATA_CLEAN/"2024_B2CC_B5_F10070.root",
    DATA_CLEAN/"2024_B2CC_B5_F10072.root", DATA_CLEAN/"2024_B2CC_B5_F10073.root", DATA_CLEAN/"2024_B2CC_B5_F10074.root",
    DATA_CLEAN/"2024_B2CC_B5_F10075.root", DATA_CLEAN/"2024_B2CC_B5_F10077.root", DATA_CLEAN/"2024_B2CC_B5_F10082.root",
    DATA_CLEAN/"2024_B2CC_B5_F10084.root", DATA_CLEAN/"2024_B2CC_B5_F10086.root", DATA_CLEAN/"2024_B2CC_B5_F10087.root",
    DATA_CLEAN/"2024_B2CC_B5_F10090.root", DATA_CLEAN/"2024_B2CC_B5_F10091.root", DATA_CLEAN/"2024_B2CC_B5_F10092.root",
    DATA_CLEAN/"2024_B2CC_B5_F10093.root", DATA_CLEAN/"2024_B2CC_B5_F10094.root", DATA_CLEAN/"2024_B2CC_B5_F10095.root",
    DATA_CLEAN/"2024_B2CC_B5_F10096.root", DATA_CLEAN/"2024_B2CC_B5_F10097.root", DATA_CLEAN/"2024_B2CC_B5_F10098.root",
    DATA_CLEAN/"2024_B2CC_B5_F10099.root", DATA_CLEAN/"2024_B2CC_B5_F10100.root", DATA_CLEAN/"2024_B2CC_B6_F10104.root",
    DATA_CLEAN/"2024_B2CC_B6_F10105.root", DATA_CLEAN/"2024_B2CC_B6_F10107.root", DATA_CLEAN/"2024_B2CC_B6_F10112.root",
    DATA_CLEAN/"2024_B2CC_B6_F10115.root", DATA_CLEAN/"2024_B2CC_B6_F10116.root", DATA_CLEAN/"2024_B2CC_B6_F10117.root",
    DATA_CLEAN/"2024_B2CC_B6_F10118.root", DATA_CLEAN/"2024_B2CC_B6_F10122.root", DATA_CLEAN/"2024_B2CC_B6_F10125.root",
    DATA_CLEAN/"2024_B2CC_B6_F10126.root", DATA_CLEAN/"2024_B2CC_B6_F10127.root", DATA_CLEAN/"2024_B2CC_B6_F10130.root",
    DATA_CLEAN/"2024_B2CC_B6_F10132.root", DATA_CLEAN/"2024_B2CC_B6_F10134.root", DATA_CLEAN/"2024_B2CC_B6_F10137.root",
    DATA_CLEAN/"2024_B2CC_B6_F10138.root", DATA_CLEAN/"2024_B2CC_B7_F10197.root", DATA_CLEAN/"2024_B2CC_B7_F10199.root",
    DATA_CLEAN/"2024_B2CC_B7_F10200.root", DATA_CLEAN/"2024_B2CC_B7_F10201.root", DATA_CLEAN/"2024_B2CC_B7_F10204.root",
    DATA_CLEAN/"2024_B2CC_B7_F10206.root", DATA_CLEAN/"2024_B2CC_B7_F10207.root", DATA_CLEAN/"2024_B2CC_B7_F10208.root",
    DATA_CLEAN/"2024_B2CC_B7_F10209.root", DATA_CLEAN/"2024_B2CC_B7_F10210.root", DATA_CLEAN/"2024_B2CC_B7_F10211.root",
    DATA_CLEAN/"2024_B2CC_B7_F10213.root", DATA_CLEAN/"2024_B2CC_B8_F10214.root", DATA_CLEAN/"2024_B2CC_B8_F10215.root",
    DATA_CLEAN/"2024_B2CC_B8_F10216.root", DATA_CLEAN/"2024_B2CC_B8_F10218.root", DATA_CLEAN/"2024_B2CC_B8_F10219.root",
    DATA_CLEAN/"2024_B2CC_B8_F10222.root", DATA_CLEAN/"2024_B2CC_B8_F10223.root", DATA_CLEAN/"2024_B2CC_B8_F10225.root",
    DATA_CLEAN/"2024_B2CC_B8_F10226.root", DATA_CLEAN/"2024_B2CC_B8_F10230.root", DATA_CLEAN/"2024_B2CC_B8_F10232.root"
]

# Observable: B+ mass variable MeV/c² range
x = r.RooRealVar("Bp_DTF_OwnPV_MASS", "B^{+} mass", 5200, 5450)   

# B+ → J/ψK+ signal parameters (main peak around 5279 MeV)
mean = r.RooRealVar("mean", "mean", 5279.38, 5250, 5300)

# Background model
tau = r.RooRealVar("tau", "Decay constant", -0.00107319, -1, 0) 
background = r.RooExponential("background", "Exponential background", x, tau)

# Crystal Ball 1 parameters
alpha = r.RooRealVar("alpha", "alpha", 1.59501, 0.1, 2)
n = r.RooRealVar("n", "n", 2.75673, 0.5, 5)
cb_sigma = r.RooRealVar("cb_sigma", "cb_sigma", 7.2753, 5, 10)
crystal_ball = r.RooCBShape("crystal_ball", "Crystal ball PDF", x, mean, cb_sigma, alpha, n)

# Crystal Ball 2 parameters
alpha_2 = r.RooRealVar("alpha2", "alpha2", -1.29179, -1.5, -0.1)
n_2 = r.RooRealVar("n2", "n2", 5.29, 0.5, 10)
cb_sigma_2 = r.RooRealVar("cb_sigma2", "cb_sigma2", 8.21279, 5, 10)
crystal_ball_2 = r.RooCBShape("crystal_ball2", "Crystal ball PDF 2", x, mean, cb_sigma_2, alpha_2, n_2)

frac_cb_2 = r.RooRealVar("frac_cb_2", "Fraction of crystal ball 2", 0.5, 0.0, 1.0)

# Parameter constraints: fix shape parameters, float yields and resolutions
mean.setConstant(False)          # allow mean to float
alpha.setConstant(True)          # fix tail parameter
n.setConstant(True)              # fix tail parameter
cb_sigma.setConstant(True)       # fix resolution
tau.setConstant(False)           # allow background slope to float
alpha_2.setConstant(True)        # fix tail parameter
n_2.setConstant(True)            # fix tail parameter
cb_sigma_2.setConstant(False)    # allow resolution to float
frac_cb_2.setConstant(False)     # allow CB fraction to float

# Helper functions for branch management
def make_branch(tree, name, var_type='d'):
    """Helper function to create a branch with array buffer"""
    if var_type == 'd':
        buf = array('d', [0.])
        tree.Branch(name, buf, f"{name}/D")
    elif var_type == 'i':
        buf = array('i', [0])
        tree.Branch(name, buf, f"{name}/I")
    return buf

def setaddr(tree, name, buf): 
    """Helper function to set branch addresses safely"""
    branch = tree.GetBranch(name)
    if branch: 
        branch.SetAddress(buf)

# Main fitting loop over input files
for fname in files:

    print(f"\n▶ fitting fill file {fname}")

    # Open file and get tree, UPDATE keeps existing content
    f = r.TFile.Open(str(fname), "UPDATE")        
    tree = f.Get("ST-b2cc")
    if not tree:
        print("ST-b2cc tree not found, skipping");  f.Close();  continue

    # Load data into RooDataSet
    data = r.RooDataSet("data", "data", r.RooArgSet(x), r.RooFit.Import(tree))
    Nevt = data.numEntries()
    print(f"entries in tree = {Nevt}")

    # Define yield variables with initial estimates
    nsig = r.RooRealVar("nsig", "yield sig", Nevt * 0.25, Nevt * 0.05, Nevt * 0.4)   # ~25% signal
    nbkg = r.RooRealVar("nbkg", "yield bkg", Nevt * 0.75, Nevt * 0.6, Nevt * 0.95)   # ~75% background
    ntot = r.RooFormulaVar("ntot","@0+@1", r.RooArgList(nsig, nbkg))
    
    # Construct composite PDF
    sig_pdf = r.RooAddPdf("sig","", r.RooArgList(crystal_ball_2, crystal_ball,),
                                    r.RooArgList(frac_cb_2))

    # Total model: signal + background
    model = r.RooAddPdf("model","sig+bkg", r.RooArgList(sig_pdf, background),
                                         r.RooArgList(nsig,    nbkg))

    # Perform fit
    fit_res = model.fitTo(data, r.RooFit.Save(True), r.RooFit.Strategy(2))

    # Print fit results
    print("\n=== fitted yields ===")
    print(f"nsig  = {nsig.getVal():.0f} ± {nsig.getError():.0f}")
    print(f"nbkg  = {nbkg.getVal():.0f} ± {nbkg.getError():.0f}")
    print(f"ntot  = {ntot.getVal():.0f} (derived)")
    print(frac_cb_2.getVal())
    
    # Error propagation with uncertainties package
    sig_u = ufloat(nsig.getVal(), nsig.getError()) 
    bkg_u = ufloat(nbkg.getVal(), nbkg.getError())
    frac_bkg = bkg_u / (sig_u + bkg_u)

    print(f"background fraction = {frac_bkg:.6f}")
    print(f"fit status = {fit_res.status()}, covQual = {fit_res.covQual()}")

    # Store results in TTree branches
    f.cd()
    res_tree = f.Get("fit_results")
    
    # Create results tree if it doesn't exist
    if not res_tree:
        res_tree = r.TTree("fit_results", "Mass-fit results per fill")
        
        # Create branches using helper function
        br_mean           = make_branch(res_tree, "mean")
        br_mean_e         = make_branch(res_tree, "mean_err")
        br_alpha          = make_branch(res_tree, "alpha1")
        br_alpha_e        = make_branch(res_tree, "alpha1_err")
        br_n              = make_branch(res_tree, "n1")
        br_n_e            = make_branch(res_tree, "n1_err")
        br_cb_sigma       = make_branch(res_tree, "cb_sigma1")
        br_cb_sigma_e     = make_branch(res_tree, "cb_sigma1_err")
        br_tau            = make_branch(res_tree, "tau")
        br_tau_e          = make_branch(res_tree, "tau_err")
        br_alpha_2        = make_branch(res_tree, "alpha2")
        br_alpha_2_e      = make_branch(res_tree, "alpha2_err")
        br_n_2            = make_branch(res_tree, "n2")
        br_n_2_e          = make_branch(res_tree, "n2_err")
        br_cb_sigma_2     = make_branch(res_tree, "cb_sigma2")
        br_cb_sigma_2_e   = make_branch(res_tree, "cb_sigma2_err")
        br_frac_cb_2      = make_branch(res_tree, "frac_cb_2")
        br_frac_cb_2_e    = make_branch(res_tree, "frac_cb_2_err")
        br_Nevt           = make_branch(res_tree, "Nevt")          
        br_nsig           = make_branch(res_tree, "nsig")
        br_nsig_e         = make_branch(res_tree, "nsig_err")
        br_nbkg           = make_branch(res_tree, "nbkg")
        br_nbkg_e         = make_branch(res_tree, "nbkg_err")
        br_ntot           = make_branch(res_tree, "ntot")
        br_ntot_e         = make_branch(res_tree, "ntot_err")
        
        # Integer branches for fit quality
        br_status         = make_branch(res_tree, "status", 'i')
        br_covqual        = make_branch(res_tree, "covqual", 'i')
        
    else:
        # Tree exists: create arrays and set branch addresses using helper functions
        br_mean           = array('d', [0.]); br_mean_e         = array('d', [0.])
        br_alpha          = array('d', [0.]); br_alpha_e        = array('d', [0.])
        br_n              = array('d', [0.]); br_n_e            = array('d', [0.])
        br_cb_sigma       = array('d', [0.]); br_cb_sigma_e     = array('d', [0.])
        br_tau            = array('d', [0.]); br_tau_e          = array('d', [0.])
        br_alpha_2        = array('d', [0.]); br_alpha_2_e      = array('d', [0.])
        br_n_2            = array('d', [0.]); br_n_2_e          = array('d', [0.])
        br_cb_sigma_2     = array('d', [0.]); br_cb_sigma_2_e   = array('d', [0.])
        br_frac_cb_2      = array('d', [0.]); br_frac_cb_2_e    = array('d', [0.])
        br_Nevt           = array('d', [0.])
        br_nsig           = array('d', [0.]); br_nsig_e         = array('d', [0.])
        br_nbkg           = array('d', [0.]); br_nbkg_e         = array('d', [0.])
        br_ntot           = array('d', [0.]); br_ntot_e         = array('d', [0.])
        br_status         = array('i', [0]); br_covqual        = array('i', [0])

        # Set all branch addresses using dictionary approach with helper function
        branch_mapping = {
            "mean": br_mean, "mean_err": br_mean_e,
            "alpha1": br_alpha, "alpha1_err": br_alpha_e,
            "n1": br_n, "n1_err": br_n_e,
            "cb_sigma1": br_cb_sigma, "cb_sigma1_err": br_cb_sigma_e,
            "tau": br_tau, "tau_err": br_tau_e,
            "alpha2": br_alpha_2, "alpha2_err": br_alpha_2_e,
            "n2": br_n_2, "n2_err": br_n_2_e,
            "cb_sigma2": br_cb_sigma_2, "cb_sigma2_err": br_cb_sigma_2_e,
            "frac_cb_2": br_frac_cb_2, "frac_cb_2_err": br_frac_cb_2_e,
            "Nevt": br_Nevt,
            "nsig": br_nsig, "nsig_err": br_nsig_e,
            "nbkg": br_nbkg, "nbkg_err": br_nbkg_e,
            "ntot": br_ntot, "ntot_err": br_ntot_e,
            "status": br_status, "covqual": br_covqual
        }
        
        for branch_name, buffer in branch_mapping.items():
            setaddr(res_tree, branch_name, buffer)

    # 5) fill the branch values

    br_mean[0]            = mean.getVal()
    br_mean_e[0]          = mean.getError()
    br_alpha[0]           = alpha.getVal()
    br_alpha_e[0]         = alpha.getError()
    br_n[0]               = n.getVal()
    br_n_e[0]             = n.getError()
    br_cb_sigma[0]        = cb_sigma.getVal()
    br_cb_sigma_e[0]      = cb_sigma.getError()
    br_tau[0]             = tau.getVal()
    br_tau_e[0]           = tau.getError()
    br_alpha_2[0]         = alpha_2.getVal()
    br_alpha_2_e[0]       = alpha_2.getError()
    br_n_2[0]             = n_2.getVal()
    br_n_2_e[0]           = n_2.getError()
    br_cb_sigma_2[0]      = cb_sigma_2.getVal()
    br_cb_sigma_2_e[0]    = cb_sigma_2.getError()
    br_frac_cb_2[0]       = frac_cb_2.getVal()
    br_frac_cb_2_e[0]     = frac_cb_2.getError()
    br_Nevt[0]            = Nevt
    br_nsig[0]            = nsig.getVal();        br_nsig_e[0]   = nsig.getError()
    br_nbkg[0]            = nbkg.getVal();        br_nbkg_e[0]   = nbkg.getError()
    ntot_u                = sig_u + bkg_u         
    br_ntot[0]            = ntot_u.n;             br_ntot_e[0]   = ntot_u.s
    br_status[0]          = fit_res.status();     br_covqual[0]  = fit_res.covQual()

    res_tree.Fill()
    res_tree.Write("", r.TObject.kOverwrite)

    # 5) make & save the canvas (linear + log)
    canvas = r.TCanvas(f"canvas_{fname.stem}", "JpsiK mass fit", 800, 600)
    frame = x.frame(r.RooFit.Title(f"JpsiK mass fit: {fname.stem}"))
    
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kWhite), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.Components(background), r.RooFit.FillColor(r.kGreen + 2), r.RooFit.FillStyle(3001), r.RooFit.DrawOption("F"), r.RooFit.LineColor(r.kGreen + 2), r.RooFit.LineStyle(r.kDashed))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kBlack), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.LineColor(r.kRed), r.RooFit.Name("total_curve"))
    model.plotOn(frame, r.RooFit.Components(crystal_ball), r.RooFit.LineStyle(2), r.RooFit.LineColor(r.kBlue))
    model.plotOn(frame, r.RooFit.Components(crystal_ball_2), r.RooFit.LineStyle(3), r.RooFit.LineColor(r.kBlue))

    # Format and draw the plot
    frame.SetTitle(f"B+ -> JpsiK Mass Fit: {fname.stem}")
    frame.GetXaxis().SetTitle("M(B^{+})  [MeV/c^{2}]")
    frame.GetXaxis().CenterTitle(True)
    frame.GetYaxis().CenterTitle(True)
    frame.Draw()
    
    # Add legend for plot components
    legend = r.TLegend(0.72, 0.60, 0.98, 0.88)
    legend.SetTextSize(0.025)
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.AddEntry(frame.findObject("total_curve"), "Total fit", "l")
    legend.AddEntry(frame.findObject("h_data"), "Data", "lep")
    legend.AddEntry(0, "Background", "l")  
    legend.Draw()

    # Calculate automatic y-axis minimum for log scale
    x_min_bg = 5400 
    x_max_bg = 5450
    n_bins = 100
    bin_width = (x.getMax() - x.getMin()) / n_bins
    n_bg = data.reduce(f"{x.GetName()} >= {x_min_bg} && {x.GetName()} < {x_max_bg}").numEntries()
    ymin = max(1, n_bg / ((x_max_bg - x_min_bg) / bin_width))

    # Apply log scale and save PNG
    frame.SetMinimum(ymin * 0.8)
    canvas.SetLogy()
    canvas.Write(f"canvas_{fname.stem}_log")

    # Save log-scale PNG to output directory
    log_plot_path = DATA_CLEAN / f"jpsik_fit_{fname.stem}_log.png"
    canvas.SaveAs(str(log_plot_path))
    print(f"saved log-scale plot: {log_plot_path}")

    # Save results and clean up
    res_tree.Write("", r.TObject.kOverwrite)
    f.Close()

    print(f"Completed processing {fname.name}\n")


In [23]:

# Purge fit outputs from B2OC ROOT files safely.
# - removes TTree "fit_results"
# - removes all TCanvas objects written by the fitter (canvas_<stem> and canvas_<stem>_log)
# - deletes exported PDFs named "jpsiK_fit_<stem>.pdf" in DATA_CLEAN
#
# Set DRY_RUN=True to preview actions without modifying files.

import ROOT as r
from pathlib import Path
import re

r.gROOT.SetBatch(True)

# ------------- config -------------
DRY_RUN = False

# Ensure DATA_CLEAN is defined in your session; otherwise set it here:
# DATA_CLEAN = Path("/path/to/data/processed_clean_bp_p")

files = [
    DATA_CLEAN/"2024_B2OC_B5_F10090.root", DATA_CLEAN/"2024_B2OC_B5_F10059.root", DATA_CLEAN/"2024_B2OC_B5_F10061.root",
    DATA_CLEAN/"2024_B2OC_B5_F10066.root", DATA_CLEAN/"2024_B2OC_B5_F10069.root", DATA_CLEAN/"2024_B2OC_B5_F10070.root",
    DATA_CLEAN/"2024_B2OC_B5_F10072.root", DATA_CLEAN/"2024_B2OC_B5_F10073.root", DATA_CLEAN/"2024_B2OC_B5_F10074.root",
    DATA_CLEAN/"2024_B2OC_B5_F10075.root", DATA_CLEAN/"2024_B2OC_B5_F10077.root", DATA_CLEAN/"2024_B2OC_B5_F10082.root",
    DATA_CLEAN/"2024_B2OC_B5_F10084.root", DATA_CLEAN/"2024_B2OC_B5_F10086.root", DATA_CLEAN/"2024_B2OC_B5_F10087.root",
    DATA_CLEAN/"2024_B2OC_B5_F10090.root", DATA_CLEAN/"2024_B2OC_B5_F10091.root", DATA_CLEAN/"2024_B2OC_B5_F10092.root",
    DATA_CLEAN/"2024_B2OC_B5_F10093.root", DATA_CLEAN/"2024_B2OC_B5_F10094.root", DATA_CLEAN/"2024_B2OC_B5_F10095.root",
    DATA_CLEAN/"2024_B2OC_B5_F10096.root", DATA_CLEAN/"2024_B2OC_B5_F10097.root", DATA_CLEAN/"2024_B2OC_B5_F10098.root",
    DATA_CLEAN/"2024_B2OC_B5_F10099.root", DATA_CLEAN/"2024_B2OC_B5_F10100.root", DATA_CLEAN/"2024_B2OC_B6_F10104.root",
    DATA_CLEAN/"2024_B2OC_B6_F10105.root", DATA_CLEAN/"2024_B2OC_B6_F10107.root", DATA_CLEAN/"2024_B2OC_B6_F10112.root",
    DATA_CLEAN/"2024_B2OC_B6_F10115.root", DATA_CLEAN/"2024_B2OC_B6_F10116.root", DATA_CLEAN/"2024_B2OC_B6_F10117.root",
    DATA_CLEAN/"2024_B2OC_B6_F10118.root", DATA_CLEAN/"2024_B2OC_B6_F10122.root", DATA_CLEAN/"2024_B2OC_B6_F10125.root",
    DATA_CLEAN/"2024_B2OC_B6_F10126.root", DATA_CLEAN/"2024_B2OC_B6_F10127.root", DATA_CLEAN/"2024_B2OC_B6_F10130.root",
    DATA_CLEAN/"2024_B2OC_B6_F10132.root", DATA_CLEAN/"2024_B2OC_B6_F10134.root", DATA_CLEAN/"2024_B2OC_B6_F10137.root",
    DATA_CLEAN/"2024_B2OC_B6_F10138.root", DATA_CLEAN/"2024_B2OC_B7_F10197.root", DATA_CLEAN/"2024_B2OC_B7_F10199.root",
    DATA_CLEAN/"2024_B2OC_B7_F10200.root", DATA_CLEAN/"2024_B2OC_B7_F10201.root", DATA_CLEAN/"2024_B2OC_B7_F10204.root",
    DATA_CLEAN/"2024_B2OC_B7_F10206.root", DATA_CLEAN/"2024_B2OC_B7_F10207.root", DATA_CLEAN/"2024_B2OC_B7_F10208.root",
    DATA_CLEAN/"2024_B2OC_B7_F10209.root", DATA_CLEAN/"2024_B2OC_B7_F10210.root", DATA_CLEAN/"2024_B2OC_B7_F10211.root",
    DATA_CLEAN/"2024_B2OC_B7_F10213.root", DATA_CLEAN/"2024_B2OC_B8_F10214.root", DATA_CLEAN/"2024_B2OC_B8_F10215.root",
    DATA_CLEAN/"2024_B2OC_B8_F10216.root", DATA_CLEAN/"2024_B2OC_B8_F10218.root", DATA_CLEAN/"2024_B2OC_B8_F10219.root",
    DATA_CLEAN/"2024_B2OC_B8_F10222.root", DATA_CLEAN/"2024_B2OC_B8_F10223.root", DATA_CLEAN/"2024_B2OC_B8_F10225.root",
    DATA_CLEAN/"2024_B2OC_B8_F10226.root", DATA_CLEAN/"2024_B2OC_B8_F10230.root", DATA_CLEAN/"2024_B2OC_B8_F10232.root",
]

# Canvases in this pipeline were written as "canvas_<stem>" and "canvas_<stem>_log"
CANVAS_PATTERNS = [re.compile(r"^canvas_.*$")]

def should_delete_canvas(name: str) -> bool:
    return any(pat.match(name) for pat in CANVAS_PATTERNS) or name.endswith("_log")

def purge_root_file(path: Path):
    if not path.exists():
        print(f"[skip] missing file: {path}")
        return

    print(f"\n--- cleaning: {path.name} ---")
    tf = r.TFile.Open(str(path), "UPDATE")
    if not tf or tf.IsZombie():
        print(f"[warn] cannot open: {path}")
        return

    tf.cd()

    # First pass: collect names to delete (avoid segfault from mutating while iterating)
    to_delete = []

    fit_results = tf.Get("fit_results")
    if fit_results and fit_results.InheritsFrom("TTree"):
        print("  - schedule delete: TTree fit_results")
        to_delete.append("fit_results")
    else:
        print("  - no fit_results tree")

    keys = tf.GetListOfKeys()
    if keys:
        snapshot = []
        for i in range(keys.GetSize()):
            k = keys.At(i)
            if not k:
                continue
            snapshot.append((str(k.GetName()), str(k.GetClassName())))
        for name, cls in snapshot:
            if cls == "TCanvas" or should_delete_canvas(name):
                to_delete.append(name)

    if to_delete:
        for name in to_delete:
            print(f"  - delete key: {name}")
            if not DRY_RUN:
                tf.Delete(f"{name};*")
    else:
        print("  - no matching canvases/trees")

    if not DRY_RUN:
        tf.Write("", r.TObject.kOverwrite)
    tf.Close()

    # Remove exported PDF produced by this fitter
    pdf_path = DATA_CLEAN / f"jpsiK_fit_{path.stem}.pdf"
    if pdf_path.exists():
        print(f"  - remove PDF: {pdf_path.name}")
        if not DRY_RUN:
            try:
                pdf_path.unlink()
            except Exception as e:
                print(f"    [warn] failed to remove PDF: {e}")
    else:
        print("  - no exported PDF")

def main():
    total = len(files)
    print(f"Cleaning {total} B2OC files (DRY_RUN={DRY_RUN})")
    for fpath in files:
        purge_root_file(Path(fpath))
    print("Done.")

if __name__ == "__main__":
    main()


Cleaning 66 B2OC files (DRY_RUN=False)

--- cleaning: 2024_B2OC_B5_F10090.root ---
  - schedule delete: TTree fit_results
  - delete key: fit_results
  - delete key: canvas_2024_B2OC_B5_F10090_log
  - no exported PDF

--- cleaning: 2024_B2OC_B5_F10059.root ---
  - schedule delete: TTree fit_results
  - delete key: fit_results
  - delete key: canvas_2024_B2OC_B5_F10059_log
  - no exported PDF

--- cleaning: 2024_B2OC_B5_F10061.root ---
  - schedule delete: TTree fit_results
  - delete key: fit_results
  - delete key: canvas_2024_B2OC_B5_F10061_log
  - no exported PDF

--- cleaning: 2024_B2OC_B5_F10066.root ---
  - schedule delete: TTree fit_results
  - delete key: fit_results
  - delete key: canvas_2024_B2OC_B5_F10066_log
  - no exported PDF

--- cleaning: 2024_B2OC_B5_F10069.root ---
  - schedule delete: TTree fit_results
  - delete key: fit_results
  - delete key: canvas_2024_B2OC_B5_F10069_log
  - no exported PDF

--- cleaning: 2024_B2OC_B5_F10070.root ---
  - schedule delete: TTree 